Q1.  
[10] Eigenfaces 
1. data partition 하기
2. S=(1/N)AAT : train data에 PCA 적용
 including: the eigenvectors, the eigenvalues, and the mean image, how many eigenvectors with non-zero eigenvalues are obtained and how many eigenvectors are to be used for face recognition.
3. S=(1/N)ATA : train data에 PCA 적용, 2와 비교
 including: if the eigenvectors and eigenvalues obtained are identical, what are the pros/cons of each method. (Show respective measurements)

[15] Application of Eigenfaces 
1. face image reconstruction 
 변수: varying the number of bases 
 including: if the reconstruction error (or the distortion measure) obtained is same as in the theory, how good the reconstruction / results are for at least 3 images of your choice (e.g. from both the training and testing dataset). 
2. PCA-based face recognition 
 변수: 아키텍쳐에서 사용된 parameter를 달리하여 실험할 것
 including: accuracy, example success and failure cases, the confusion matrices, time/memory

In [9]:
import sys
sys.path.append('../')

from dataset.data import split_data
from question1 import eigen

import numpy as np
import matplotlib.pyplot as plt
import time
import psutil

In [11]:
# 1. split data using data.py function
train_data, train_label, test_data, test_label = split_data(data_path="../dataset/face.mat")
mean_face = np.mean(train_data, axis=1).reshape(-1, 1)

In [ ]:
# 2. 